In [1]:
import uproot
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define file locations
FILE_LOC = "/fast_scratch_1/jbohm/cell_particle_deposit_learning/delta/delta_root_files/delta_full.root"
CELL_GEO_FILE_LOC = "/data/atlas/data/rho_delta/rho_small.root"

# Open the ROOT files
events = uproot.open(FILE_LOC + ":EventTree")
cell_geo_tree = uproot.open(CELL_GEO_FILE_LOC + ":CellGeo")

# Load the number of truth particles per event
n_truth_part = events["nTruthPart"].array()

# Find the first event with 8 or more truth particles
num_particles = 8
first_index = ak.firsts(ak.where(n_truth_part >= num_particles))

if not ak.is_valid(first_index):
    print(f"No event with {num_particles} or more truth particles was found.")
else:
    print(f"Event {first_index} has {n_truth_part[first_index]} truth particles.")

    # Load data for the specific event
    event_data = events.arrays(entry_start=first_index[0], entry_stop=first_index[0] + 1, library="np")
    cell_geo_data = cell_geo_tree.arrays(library='np')

    # Function to convert eta, phi to x, y, z coordinates
    def eta_phi_to_xyz(eta, phi, rPerp):
        theta = 2 * np.arctan(np.exp(-eta))
        x = rPerp * np.cos(phi)
        y = rPerp * np.sin(phi)
        z = rPerp / np.tan(theta)
        return x, y, z

    # Extract eta, phi, rPerp values
    eta = cell_geo_data["cell_geo_eta"][0]
    phi = cell_geo_data["cell_geo_phi"][0]
    rPerp = cell_geo_data["cell_geo_rPerp"][0]

    # Convert eta, phi to cartesian coordinates (x, y, z)
    x, y, z = eta_phi_to_xyz(eta, phi, rPerp)

    # Create a 3D plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Scatter plot using x, y, z coordinates and a single color for all points
    scatter = ax.scatter(x, y, z, color='b')

    # Set labels
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')

    # Show the plot
    plt.show()


Event [161730] has [8] truth particles.


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'Array'